# Secure Message Hiding with Python - Encryption and Steganography

## Overview
This notebook demonstrates how to securely hide a secret message inside an image using a combination of encryption and steganography. The process involves:
1. Encrypting the message using AES (Advanced Encryption Standard).
2. Hiding the encrypted message inside an image using Least Significant Bit (LSB) steganography with the `Stegano` library.
3. Extracting and decrypting the hidden message to reveal the original content.

## Why Combine Encryption and Steganography?
- **Encryption** scrambles the message, making it unreadable to unauthorized parties, but doesn't hide its presence.
- **Steganography** hides the message within another medium, such as an image, but alone, it doesn't encrypt the data.

By combining both techniques, we enhance the security of the message, making it both hidden and unreadable without the decryption key.

## Use Case
This approach can be used in secure communication where messages need to be transmitted secretly without drawing attention, such as in confidential communications, digital rights management, or watermarking.


In [ ]:
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from stegano import lsb  
import base64
from os import urandom


### Explanation:
- **Cryptography Library:** Provides functions for AES encryption and decryption, including secure key generation.
- **Stegano LSB Module:** Implements steganography by hiding messages in the least significant bits of image pixels, making the changes imperceptible to the human eye.
- **Base64:** Encodes binary data as text to embed easily within images, facilitating the integration of the encrypted message with the steganographic method.
- **`urandom`:** Generates secure random numbers, particularly useful for creating salts and initialization vectors for encryption.


In [ ]:

def generate_key(password: bytes, salt: bytes):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  
        salt=salt,
        iterations=100000,  
        backend=default_backend()
    )
    return kdf.derive(password)


password = b'PASSWORD'
salt = urandom(16)  
key = generate_key(password, salt)
print("Key generated successfully.")


### Explanation:
- **Key Generation:** Uses PBKDF2 (Password-Based Key Derivation Function 2) with HMAC and SHA-256 to derive a secure key from a password.
- **Salt:** A random value that adds additional security to the key generation process, preventing attackers from using precomputed tables (rainbow tables) to crack the key.
- **AES-256 Encryption:** A strong encryption standard used widely in secure communications; the key length of 32 bytes ensures high security.


In [ ]:
def encrypt_message(message: str, key: bytes):
    padder = padding.PKCS7(algorithms.AES.block_size).padder()  
    padded_data = padder.update(message.encode()) + padder.finalize()
    
    iv = urandom(16)  # Generate a random initialization vector (IV)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())  
    encryptor = cipher.encryptor()
    encrypted_message = encryptor.update(padded_data) + encryptor.finalize()  
    
    return iv + encrypted_message  

# Example message encryption
message = "VERY SECRET MESSAGE"
encrypted_message = encrypt_message(message, key)
print("Message encrypted successfully.")


### Explanation:
- **AES Encryption:** AES (Advanced Encryption Standard) is used in CBC (Cipher Block Chaining) mode, which adds security by chaining each block of data with the previous block.
- **Padding:** Ensures the message fits the AES block size of 128 bits; without padding, the message may not be correctly encrypted.
- **IV (Initialization Vector):** A random 16-byte value that ensures the same plaintext encrypted multiple times will produce different ciphertexts, enhancing security.


In [ ]:
def hide_message_in_image(image_path: str, output_image_path: str, message: str, key: bytes):
    encrypted_message = encrypt_message(message, key)
    encoded_message = base64.b64encode(encrypted_message).decode('utf-8')  # Convert the encrypted message to a base64 string
    
    # Hide the encoded message in the image
    secret_image = lsb.hide(image_path, encoded_message)
    secret_image.save(output_image_path)
    print(f"Message hidden successfully in {output_image_path}.")

# Hide the message in an image
image_path = r"ORIGINAL IMAGE PATH"  
output_image_path = r"ENCRYPTED IMAGE PATH"  
hide_message_in_image(image_path, output_image_path, message, key)


### Explanation:
- **Base64 Encoding:** Converts the encrypted message into a text format, making it easier to embed within the image pixels.
- **Steganography with LSB (Least Significant Bit):** Hides the encoded message within the least significant bits of the image pixels, making the hidden data imperceptible to the naked eye.
- **Saving the Image:** The modified image, now containing the hidden message, is saved to a new file.


In [ ]:

def reveal_message_from_image(image_path: str, key: bytes):
    extracted_message = lsb.reveal(image_path)  
    decoded_message = base64.b64decode(extracted_message)  
    decrypted_message = decrypt_message(decoded_message, key)  
    return decrypted_message


def decrypt_message(encrypted_message: bytes, key: bytes):
    iv = encrypted_message[:16]  # Extract the IV
    encrypted_message = encrypted_message[16:]  # Extract the actual encrypted message
    
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())  # Set up the AES cipher for decryption
    decryptor = cipher.decryptor()
    decrypted_padded_message = decryptor.update(encrypted_message) + decryptor.finalize()
    
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    decrypted_message = unpadder.update(decrypted_padded_message) + unpadder.finalize()
    
    return decrypted_message.decode()


revealed_message = reveal_message_from_image(output_image_path, key)
print(f"Revealed and decrypted message: {revealed_message}")


### Explanation:
- **Revealing the Message:** The `reveal()` function extracts the hidden base64-encoded message from the modified image.
- **Decoding and Decrypting:** The extracted message is decoded from base64 and decrypted using the original AES key and IV, restoring the original message to its readable form.


------------------------------------------------------------------------------------------------------------------------------------------------
## Pros:
1. **Dual Layer Security**: Combining encryption with steganography significantly increases message security by making it both hidden and encrypted.
2. **Ease of Use**: The Stegano library simplifies the process of embedding messages in images without needing extensive steganography knowledge.
3. **Untraceable Communication**: The technique makes messages less detectable compared to plain encryption.

## Cons:
1. **Image Quality Sensitivity**: LSB steganography can be fragile; modifying the image (resizing, compressing) may destroy the hidden message.
2. **Limited Capacity**: The amount of data that can be hidden depends on the image size and type; large messages may not fit or degrade the image noticeably.
3. **Steganalysis Risk**: Advanced tools can detect alterations in images, potentially revealing the presence of hidden data.

## Potential Improvements:
1. **Enhanced Encryption**: Consider using authenticated encryption modes (e.g., AES-GCM) that provide integrity checks, ensuring the message hasn't been tampered with.
2. **Dynamic Steganography**: Use adaptive methods that vary the embedding strategy based on image content

3. **Error Checking and Correction**: Implement techniques to recover or correct hidden messages that may be partially damaged due to image modifications.

This combination of encryption and steganography creates a robust, secure way to hide sensitive information in plain sight, with potential enhancements making it even more resilient against detection and data loss.
